# Chapter 10 The Factory

The factory is the last design pattern covered. The main objective of the factory is to provide a class that spits objects when called, and transforms strings into objects. With that it is able to provide an easy interface and also develop the open-closed principle.


## The main class


RegisterPayOff allow us to register new payoffs and CreatePayOff searches into the dictionary TheCratorFunctions for a constructor for this payoff and calls it. We use a dictionary to store creator functions (aka constructors) because it allow us to save unique pairs between a string (the key) and a function (the value).

In [2]:
class PayOffFactory:
    def __init__(self):
        self.__TheCreatorFunctions = {}        
    
    def __del__(self):
        del self
    
    def RegisterPayOff(self, PayOffId, CreatorFunction):
        self.__TheCreatorFunctions[PayOffId] = CreatorFunction
        
    def CreatePayOff(self, PayOffId, Strike):
        if PayOffId not in self.__TheCreatorFunctions.keys():
            print(f'{PayOffId} is an unknown payoff')
            return None
        else:
            return self.__TheCreatorFunctions[PayOffId](Strike)   

The initial code adds a template class for handling the registrations, the code looks like this:

In [3]:
class PayOffHelper:    
    
    def __init__(self,PayOffId,PayOff):
        self.PayOff = PayOff        
        thePayOffFactory.RegisterPayOff(PayOffId,self.Create)
        

    def Create(self, Strike):
        return self.PayOff(Strike)
    

However this can be confusing and a little bit of a mess, this construction makes sense in strong typed languages such as C++ but in python this does not make sense, and it is way simpler to just call the method, since it is not needed the PayOffHelper (that in the C++ code was a template class). In C++ the template PayOffHelper allows to register payoffs without knowing the type (something that is not possible in the method of the Factory class), in python knowing the type is not needed, thus it makes no sense to really use it.


### Using thePayOffFactory

We copy all the code for the PayOffs.

In [4]:
class PayOff:
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

class PayOffBridge:        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
        
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)

class PayOffPut(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(self.__Strike - Spot,0) 
    
class PayOffForward(PayOff,PayOffBridge):
    def __init__(self, Strike):
        self.__Strike = Strike
        
    def __call__(self, Spot):
        return Spot - Strike

In [5]:
global thePayOffFactory  # We just the global keyword to be accesible from anywhere
thePayOffFactory  = PayOffFactory()

Registering objects using the PayOffHelper

In [6]:
RegisterCall = PayOffHelper("call",PayOffCall)
RegisterPut = PayOffHelper("put",PayOffPut)

Now we use it for a very simple example.

In [7]:
Strike = 2
name = "call"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


Now we will do the same but registering it with the method of the factory.

In [8]:
del thePayOffFactory

In [9]:
global thePayOffFactory  # We just the global keyword to be accesible from anywhere
thePayOffFactory  = PayOffFactory()
thePayOffFactory.RegisterPayOff("call",PayOffCall)
thePayOffFactory.RegisterPayOff("put",PayOffPut)

In [10]:
Strike = 2
name = "call"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


So we can see that the result is the same.

### Exercises

**Exercise 10.1** Write a straddle class and register it with the factory.

The payoff of a straddle is the absolute difference between the strike and the spot.

Let's implement the class.

In [11]:
class PayOffStraddle(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return abs(self.__Strike - Spot) 

Now we will register it.

In [12]:
thePayOffFactory.RegisterPayOff("straddle",PayOffStraddle)

And then we try some easy examples.

In [13]:
Strike = 2
name = "straddle"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


In [14]:
Strike = 4
name = "straddle"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


**Exercise 10.2** Our class cannot handle a double digital as it needs two strikes. Work out a solution that will handle options with multiple parameters.

We can do several things for that. One way is to rewrite it so we input a dictionary and then the payoff extracts the necessary data from the dictionary. 

Other way of doing it is just put *args, ** kwargs, this python syntax allow us to input all the variables we want without having to previously specify them. We will take the second approach since it does not require to rewritte the class.

In [15]:
class PayOffFactory:
    def __init__(self):
        self.__TheCreatorFunctions = {}        
    
    def __del__(self):
        del self
    
    def RegisterPayOff(self, PayOffId, CreatorFunction):
        self.__TheCreatorFunctions[PayOffId] = CreatorFunction
        
    def CreatePayOff(self, PayOffId, *args, **kwargs):
        if PayOffId not in self.__TheCreatorFunctions.keys():
            print(f'{PayOffId} is an unknown payoff')
            return None
        else:
            return self.__TheCreatorFunctions[PayOffId](*args, **kwargs)   

We will see if it works for a plain vanilla call.

In [16]:
del thePayOffFactory

In [17]:
thePayOffFactory  = PayOffFactory()
thePayOffFactory.RegisterPayOff("call",PayOffCall)

In [18]:
Strike = 2
name = "call"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


Now we will use it with a double digital.

In [19]:
class PayOffDoubleDigital(PayOff,PayOffBridge):
    def __init__(self, LowerLevel_, UpperLevel_):
        self.__LowerLevel = LowerLevel_
        self.__UpperLevel = UpperLevel_
        
    def __call__(self,Spot):
        if Spot <= self.__LowerLevel or Spot >=  self.__UpperLevel:
            return 0
        else:
            return 1           

In [20]:
thePayOffFactory.RegisterPayOff("doubleDigital",PayOffDoubleDigital)

In [21]:
UpperLevel = 2
LowerLevel = 5
name = "doubleDigital"
Spot = 4
thePayOFF = thePayOffFactory.CreatePayOff(name,UpperLevel,LowerLevel)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


**Exercise 10.3** Integrate the factory with a Monte Carlo routine.

Finally we can easily integrate it with a Monte Carlo routine.

We will copy all the code we need (that we have developed in the previous chapters).

In [22]:
import numpy as np
from copy import deepcopy
from random import random
from math import exp, sqrt, log

class StatisticsMC:
    def __init__(self):
        pass
    
    def GetResultsSoFar(self):
        pass
    
    def DumpOneResult(self,result):
        pass        
  
    def clone(self):
        pass
    
    def __del__(self):
        pass   
    
class StatisticsMean(StatisticsMC):
    def __init__(self):
        self.__RunningSum = 0
        self.__PathsDone = 0
        
    def GetResultsSoFar(self): 
        Results = [[0]]
        Results[0][0] = self.__RunningSum / self.__PathsDone
        return Results
    
    def DumpOneResult(self,result):
        self.__PathsDone += 1
        self.__RunningSum += result        
        
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self
        
class ConvergenceTable(StatisticsMC):
    def __init__(self, Inner_):
        self.__Inner = Inner_
        self.__ResultsSoFar = []
        self.__StoppingPoint = 2
        self.__PathsDone = 0
    
    def clone(self):
        return deepcopy(self)
    
    def DumpOneResult(self,result):
        self.__Inner.DumpOneResult(result)
        self.__PathsDone += 1
        
        
        if self.__PathsDone == self.__StoppingPoint:
            self.__StoppingPoint *=2
            thisResult = self.__Inner.GetResultsSoFar()            
            
            for i in range(len(thisResult)):
                thisResult[i].append(self.__PathsDone)
                self.__ResultsSoFar.append(thisResult[i]) 

            
    
    def GetResultsSoFar(self):
        
        tmp = self.__ResultsSoFar 
        
        if self.__PathsDone*2 != self.__StoppingPoint:
            thisResult = self.__Inner.GetResultsSoFar()
            
            for i in range(len(thisResult)):
                thisResult[i].append(self.__PathsDone)
                tmp.append(thisResult[i]) 
                
        return tmp
                
        
class ParametersInner:
    def __init__(self):
        pass
    
    def Integral(self,time1, time2):
        pass
    
    def IntegralSquare(self,time1,time2):
        pass

class Parameters:    
    def RootMeanSquare(self,time1,time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def Mean(self,time1, time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def copy(self,original):
        from copy import deepcopy
        self = deepcopy(InnerPayOff) 
        
    def __del__(self):
        del self    
        
class ParametersConstant(ParametersInner,Parameters):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare   

class Generator:
    def __init__(self,Dimensionality_ = 1):
        self.__Dimensionality = Dimensionality_
        
    def ResetDimensionality(self,Dimensionality_):
        self.__Dimensionality = Dimensionality_
    
    def GetGaussians(self):
        Variates = np.random.normal(size = self.__Dimensionality)
        return Variates
    
    
class VanillaOption:
    def __init__(self, ThePayOff_, Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self, Spot):
        return self.__ThePayOff(Spot)
    
def SimpleMonteCarlo(TheOption,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths,
                     gatherer,
                     generator):
    
    Expiry = TheOption.GetExpiry()    
    variance = Vol.IntegralSquare(0,Expiry)
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r.Integral(0,Expiry) + itoCorrection)
    discounting = exp(-r.Integral(0,Expiry))
    for i in range(NumberOfPaths):
        thisGaussian = generator.GetGaussians()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = TheOption.OptionPayOff(thisSpot)
        gatherer.DumpOneResult(thisPayoff*discounting)

We eliminate the previous factory.

In [23]:
del thePayOffFactory

We create the new factory and register the call payoff.

In [24]:
thePayOffFactory  = PayOffFactory()
thePayOffFactory.RegisterPayOff("call",PayOffCall)

We set our variables.

In [25]:
Strike = 40
Spot = 42
name = "call"
r_ = 0.1
Vol_ = 0.2
NumberOfPaths = 10000
Maturity = 0.5

We run the all the code.

In [26]:
gatherer = StatisticsMean()
gathererTwo = ConvergenceTable(gatherer)
generator = Generator()
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:

    TheOption = VanillaOption(thePayOFF,Maturity)
    r = ParametersConstant(r_)
    Vol = ParametersConstant(Vol_)
    SimpleMonteCarlo(TheOption,Spot,Vol,r,NumberOfPaths, gathererTwo, generator)
    results = gathererTwo.GetResultsSoFar()
    print(f'Call price for 10000 paths: ')
    for i in range(len(results)):
        for j in range(len(results[i])):
            print(results[i][j])
else:
    print('Incorrect PayOff')

The payoff is: 2
Call price for 10000 paths: 
4.004910672086012
2
5.145002070939581
4
5.614216923908329
8
3.480848884874448
16
4.013585523228452
32
4.580557537591412
64
4.894159632796665
128
5.033604401264067
256
5.256957455873575
512
4.885656670072443
1024
4.838862747675183
2048
4.849347045770759
4096
4.778877225205469
8192
4.75831999793881
10000


We could have implemented the same but putting the factory inside the montecarlo, and then just sending a string and other specific parameters of the payoff, using the  *args,** kwargs that we have used before. However if we do it this way we will face many issues and our open closed principle might get damaged.